# FLAN-T5
# Question and Answer System

In [ ]:
!pip install langchain
!pip install langchain_community
!pip install -U langchain-huggingface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 697.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.8/401.8 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.3 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49

In [ ]:
!pip install transformers faiss-cpu sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 33.7 MB/s eta 0:00:00


In [ ]:
!pip freeze > requirement.txt

Loading Data

In [ ]:
from langchain.document_loaders import WebBaseLoader

loader = WebBaseLoader("http://jalammar.github.io/illustrated-transformer/")
data = loader.load()

Splitting Data
  Split the Document into chunks for embedding and vector storage.

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 0)
all_splits = text_splitter.split_documents(data)

Storing
  embed the contents of each document, then store the embedding and document in a vector store, with the embedding being used to index the document.

In [ ]:
# Import FAISS from Langchain Vectorstore
from langchain.vectorstores import FAISS

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
from tqdm import tqdm, trange
from transformers import AutoTokenizer

model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}

tokenizer = AutoTokenizer.from_pretrained(model_name, clean_up_tokenization_spaces=True)

hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

...

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
 # Creating a vector store
vectorstore = FAISS.from_documents(documents=all_splits, embedding=hf) ## hf are the hugging face embeddings

Retrieval

In [ ]:
question = "What are transformers?"
docs = vectorstore.similarity_search(question)
docs

[Document(metadata={'source': 'http://jalammar.github.io/illustrated-transformer/', 'title': 'The Illustrated Transformer – Jay Alammar – Visualizing machine learning one concept at a time.', 'description': 'Discussions:\nHacker News (65 points, 4 comments), Reddit r/MachineLearning (29 points, 3 comments)\n\n\nTranslations: Arabic, Chinese (Simplified) 1, Chinese (Simplified) 2, French 1, French 2, Italian, Japanese, Korean, Persian, Russian, Spanish 1, Spanish 2, Vietnamese\n\nWatch: MIT’s Deep Learning State of the Art lecture referencing this post\n\nFeatured in courses at Stanford, Harvard, MIT, Princeton, CMU and others\n\nIn the previous post, we looked at Attention – a ubiquitous method in modern deep learning models. Attention is a concept that helped improve the performance of neural machine translation applications. In this post, we will look at The Transformer – a model that uses attention to boost the speed with which these models can be trained. The Transformer outperform

Generation
  Using FLAN_T5 language model

In [ ]:
# For downloading the models from HF Hub
!pip install huggingface_hub==0.23.5 -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 402.8/402.8 kB 8.8 MB/s eta 0:00:00


In [33]:
# Set the environment variable
from google.colab import userdata
import os

# Ensure the token is correctly retrieved and assigned to the environment variable
token = userdata.get('HF_TOKEN')
if token:
  os.environ['HUGGINGFACEHUB_API_TOKEN'] = token
else:
  print("HuggingFace token not found in userdata.")

In [34]:
from langchain import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id='google/flan-t5-base',
    model_kwargs={"temperature":0, "max_length":1000, 'max_new_tokens' : 250, 'top_k' : 10, 'top_p': 0.95, 'repetition_penalty':1.03}
)

In [ ]:
#from huggingface_hub import notebook_login
#notebook_login()

In [38]:

# Define the question
question = "What is attention mechanism?"
# using the model
output = llm.invoke(question)
print(output)


a reversal of the previous state


## Chat
  - Summary Memory creation

In [39]:
from langchain.memory import ConversationSummaryMemory

In [40]:
memory = ConversationSummaryMemory(
    llm=llm,
    memory_key="chat_history",
    return_messages=True
)

<ipython-input-40-c568acba53d0>:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryMemory(


## Conversational Retrieval Chain
- chat history + new question

In [41]:
from langchain.chains import ConversationalRetrievalChain

retriever = vectorstore.as_retriever()
chat = ConversationalRetrievalChain.from_llm(
    llm,
    retriever=retriever,
    memory=memory,
    verbose=True
)

## Demonstration

In [42]:
chat("Explain self-attention")

<ipython-input-42-2782db009e86>:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chat("Explain self-attention")




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

If you’re familiar with RNNs, think of how maintaining a hidden state allows an RNN to incorporate its representation of previous words/vectors it has processed with the current one it’s processing. Self-attention is the method the Transformer uses to bake the “understanding” of other relevant words into the one we’re currently processing.

The second step in calculating self-attention is to calculate a score. Say we’re calculating the self-attention for the first word in this example, “Thinking”. We need to score each word of the input sentence against this word. The score determines how much focus to place on other parts of the input sentence as we encode a word at a certain position.

Self-Attention at a High Level
Do

{'question': 'Explain self-attention',
 'chat_history': [SystemMessage(content='', additional_kwargs={}, response_metadata={})],
 'answer': 'The Transformer uses to bake the “understanding” of other relevant words into the one we’re currently processing. The second step in calculating self-attention is to calculate a score.'}

In [43]:
chat("What is a gentler approach to transformers?")



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

system: The Transformer uses the “understanding” of other relevant words to calculate a score.
Follow Up Input: What is a gentler approach to transformers?
Standalone question:

> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Go Forth And Transform
I hope you’ve found this a useful place to start to break the ice with the major concepts of the Transformer. If you want to go deeper, I’d suggest these next steps:

2020 Update: I’ve created a “Narrated Transformer” video which is a gentler approach to the topic:

The Transformer was pr

{'question': 'What is a gentler approach to transformers?',
 'chat_history': [SystemMessage(content='The Transformer uses the “understanding” of other relevant words to calculate a score.', additional_kwargs={}, response_metadata={})],
 'answer': 'Narrated Transformer'}

In [44]:
chat("Where were transformers proposed?")



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

system: The human is a little confused about what the transformer is doing.
Follow Up Input: Where were transformers proposed?
Standalone question:

> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

2020 Update: I’ve created a “Narrated Transformer” video which is a gentler approach to the topic:

Go Forth And Transform
I hope you’ve found this a useful place to start to break the ice with the major concepts of the Transformer. If you want to go deeper, I’d suggest these next steps:

Attribution example:

Alammar, J (2018). The Illust

{'question': 'Where were transformers proposed?',
 'chat_history': [SystemMessage(content='The human is a little confused about what the transformer is doing.', additional_kwargs={}, response_metadata={})],
 'answer': 'in the paper Attention is All You Need'}

In [45]:
chat("What are the different layers in a typical Transformer model?")



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

system: The human is looking for a transformer.
Follow Up Input: What are the different layers in a typical Transformer model?
Standalone question:

> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Go Forth And Transform
I hope you’ve found this a useful place to start to break the ice with the major concepts of the Transformer. If you want to go deeper, I’d suggest these next steps:

This goes for the sub-layers of the decoder as well. If we’re to think of a Transformer of 2 stacked encoders and decoders, it would look something lik

{'question': 'What are the different layers in a typical Transformer model?',
 'chat_history': [SystemMessage(content='The human is looking for a transformer.', additional_kwargs={}, response_metadata={})],
 'answer': 'attention layer'}

In [46]:
chat("If the vocabulary is 10,000 words, what would the width of the logits vector?")



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

system: The human is looking for a transformer.
Follow Up Input: If the vocabulary is 10,000 words, what would the width of the logits vector?
Standalone question:

> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Let’s assume that our model knows 10,000 unique English words (our model’s “output vocabulary”) that it’s learned from its training dataset. This would make the logits vector 10,000 cells wide – each cell corresponding to the score of a unique word. That is how we interpret the output of the model followed by the Linear lay

{'question': 'If the vocabulary is 10,000 words, what would the width of the logits vector?',
 'chat_history': [SystemMessage(content='The human is looking for a transformer.', additional_kwargs={}, response_metadata={})],
 'answer': '10,000 cells wide'}

In [47]:
chat("Explain the training process of a Transformer network in detail")



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

system: The logits vector is 10,000 cells wide.
Follow Up Input: Explain the training process of a Transformer network in detail
Standalone question:

> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Recap Of Training
Now that we’ve covered the entire forward-pass process through a trained Transformer, it would be useful to glance at the intuition of training the model.
During training, an untrained model would go through the exact same forward pass. But since we are training it on a labeled training dataset, we can compare its outpu

{'question': 'Explain the training process of a Transformer network in detail',
 'chat_history': [SystemMessage(content='The logits vector is 10,000 cells wide.', additional_kwargs={}, response_metadata={})],
 'answer': 'Forward-pass'}